# [550. Game Play Analysis IV](https://leetcode.com/problems/game-play-analysis-iv/description/?envType=study-plan-v2&envId=top-sql-50)

Table: Activity

<pre>+--------------+---------+
| Column Name  | Type    |
+--------------+---------+
| player_id    | int     |
| device_id    | int     |
| event_date   | date    |
| games_played | int     |
+--------------+---------+</pre>
(player_id, event_date) is the primary key (combination of columns with unique values) of this table.
This table shows the activity of players of some games.
Each row is a record of a player who logged in and played a number of games (possibly 0) before logging out on someday using some device.
 

Write a solution to report the fraction of players that logged in again on the day after the day they first logged in, rounded to 2 decimal places. In other words, you need to count the number of players that logged in for at least two consecutive days starting from their first login date, then divide that number by the total number of players.

The result format is in the following example.

 

Example 1:

Input: 
Activity table:
<pre>+-----------+-----------+------------+--------------+
| player_id | device_id | event_date | games_played |
+-----------+-----------+------------+--------------+
| 1         | 2         | 2016-03-01 | 5            |
| 1         | 2         | 2016-03-02 | 6            |
| 2         | 3         | 2017-06-25 | 1            |
| 3         | 1         | 2016-03-02 | 0            |
| 3         | 4         | 2018-07-03 | 5            |
+-----------+-----------+------------+--------------+</pre>
Output: 
<pre>+-----------+
| fraction  |
+-----------+
| 0.33      |
+-----------+</pre>
Explanation: 
Only the player with id 1 logged back in after the first day he had logged in so the answer is 1/3 = 0.33

In [1]:
#pandas schema
import pandas as pd

data = [[1, 2, '2016-03-01', 5], [1, 2, '2016-03-02', 6], [2, 3, '2017-06-25', 1], [3, 1, '2016-03-02', 0],
        [3, 4, '2018-07-03', 5]]
activity = pd.DataFrame(data, columns=['player_id', 'device_id', 'event_date', 'games_played']).astype(
    {'player_id': 'Int64', 'device_id': 'Int64', 'event_date': 'datetime64[ns]', 'games_played': 'Int64'})

In [2]:
# to spark dataframe

from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

activity_df = spark.createDataFrame(activity)
activity_df.show(truncate=False)

bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
23/11/09 15:36:02 WARN Utils: Your hostname, svchost resolves to a loopback address: 127.0.1.1; using 172.18.28.34 instead (on interface eth0)
23/11/09 15:36:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/09 15:36:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+---------+---------+-------------------+------------+
|player_id|device_id|event_date         |games_played|
+---------+---------+-------------------+------------+
|1        |2        |2016-03-01 00:00:00|5           |
|1        |2        |2016-03-02 00:00:00|6           |
|2        |3        |2017-06-25 00:00:00|1           |
|3        |1        |2016-03-02 00:00:00|0           |
|3        |4        |2018-07-03 00:00:00|5           |
+---------+---------+-------------------+------------+


In [3]:
# solving in spark dataframe
from pyspark.sql import Window
import pyspark.sql.functions as F

activity_df.withColumn('next_date',
                       F.lead('event_date').over(Window.partitionBy('player_id').orderBy('event_date'))
                       ) \
    .withColumn('rn', F.row_number().over(Window.partitionBy('player_id').orderBy('event_date'))) \
    .filter(F.col('rn') == 1) \
    .agg(
    F.round(
        F.sum(F.when(F.datediff(F.col('next_date'), F.col('event_date')) == 1, 1).otherwise(0)) / F.count('player_id')
        , 2).alias('fraction')
) \
    .show()

+--------+
|fraction|
+--------+
|    0.33|
+--------+


In [4]:
# solving in Spark SQL

activity_df.createOrReplaceTempView('activity')

spark.sql('''
with tbl as (
    select 
        *,
        lead(event_date) over (partition by player_id order by event_date) as next_date,
        row_number() over (partition by player_id order by event_date) as rn
    from activity
)
select 
    round(
    sum(case when datediff(next_date,event_date)=1 then 1 else 0 end)/count(*)
    ,2) as fraction 
from tbl
where rn=1;
''').show()

+--------+
|fraction|
+--------+
|    0.33|
+--------+


In [5]:
spark.stop()